In [1]:
# Import the necessary libraries:
from bs4 import BeautifulSoup
import requests
import sqlite3
import pandas as pd
import time

# The page we need to scrape: 
TIKI_URL = 'https://tiki.vn'

# Setting up the connection: 
conn = sqlite3.connect('tiki-4.db')
cur = conn.cursor()

In [2]:
# Get the HTML content get_url()
def get_url(url):
    try:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        return(soup)
    except Exception as err:
        print('ERROR BY INSERT:',err)
get_url(TIKI_URL)

<!DOCTYPE html>
<html class="no-js" lang="vi"><head><meta charset="utf-8" class="next-head"/><meta class="next-head" content="Thỏa sức mua sắm qua mạng hàng ngàn mặt hàng sách, điện tử, đồ gia dụng, quà tặng, thời trang, làm đẹp &amp; sức khỏe tại Tiki với giá rẻ hơn và khuyến mãi hấp dẫn" name="description"/><link class="next-head" href="https://frontend.tikicdn.com/_desktop-next/static/css/_sprite.css?v=20208640474Qk1QNqcwEri1rBGnShPz" rel="stylesheet" type="text/css"/><link class="next-head" href="https://fonts.gstatic.com/" rel="dns-prefetch"/><link class="next-head" href="https://salt.tikicdn.com/" rel="dns-prefetch"/><link as="image" class="next-head" href="https://salt.tikicdn.com/cache/w584/ts/banner/f4/c7/b8/3237055cc27fabcb6f656a5f6acc3d66.jpg" rel="preload"/><link as="image" class="next-head" href="https://salt.tikicdn.com/cache/w292/ts/banner/a6/8a/de/2d1269426856cd8f1e37deb1ee4d7a97.jpg" rel="preload"/><link as="image" class="next-head" href="https://salt.tikicdn.com/cache

In [6]:
# Create table categories in the database using a function
def create_categories_table():
    query = """
        CREATE TABLE IF NOT EXISTS categories (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name VARCHAR(255),
            url TEXT, 
            parent_id INTEGER, 
            create_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """
    try:
        cur.execute(query)
        conn.commit()
    except Exception as err:
        print('ERROR BY CREATE TABLE', err)
        
create_categories_table()

In [3]:
# Instead of using a function to do CRUD on database
# Create a class Category is preferred
# attributes: name, url, parent_id
# instance method: save_into_db()
class Category:
    def __init__(self, name, url, parent_id = None, cat_id=None):
        self.name = name
        self.url = url
        self.parent_id = parent_id
        self.cat_id = cat_id
    
    def __repr__(self):
        return(f"ID:{self.cat_id}, name:{self.name}, url:{self.url}, parent:{self.parent_id}")
    
    def save_into_db(self):
        query = """
        INSERT INTO 
        categories (name,url,parent_id)
        VALUES (?, ?, ?);
        """
        val = (self.name, self.url,self.parent_id)
        try:
            cur.execute(query,val)
            self.cat_id = cur.lastrowid
            conn.commit()
        except Exception as err:
            print ('ERROR BY INSERT:',err)
        

In [4]:
# get_main_categories()
def get_main_categories(save_db=False):
    soup = get_url(TIKI_URL)

    result = []
    for a in soup.find_all('a', {'class': 'MenuItem__MenuLink-sc-181aa19-1 fKvTQu'}):
        name = a.find('span', {'class': 'text'}).text
        url = a['href']
        main_cat = Category(name, url)

        if save_db:
            main_cat.save_into_db()
        result.append(main_cat)
    return result

In [5]:
main_categories = get_main_categories(save_db=False)
main_categories

[ID:None, name:Điện Thoại - Máy Tính Bảng, url:https://tiki.vn/dien-thoai-may-tinh-bang/c1789?src=c.1789.hamburger_menu_fly_out_banner, parent:None,
 ID:None, name:Điện Tử - Điện Lạnh, url:https://tiki.vn/tivi-thiet-bi-nghe-nhin/c4221?src=c.4221.hamburger_menu_fly_out_banner, parent:None,
 ID:None, name:Phụ Kiện - Thiết Bị Số, url:https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815?src=c.1815.hamburger_menu_fly_out_banner, parent:None,
 ID:None, name:Laptop - Thiết bị IT, url:https://tiki.vn/laptop-may-vi-tinh/c1846?src=c.1846.hamburger_menu_fly_out_banner, parent:None,
 ID:None, name:Máy Ảnh - Quay Phim, url:https://tiki.vn/may-anh/c1801?src=c.1801.hamburger_menu_fly_out_banner, parent:None,
 ID:None, name:Điện Gia Dụng, url:https://tiki.vn/dien-gia-dung/c1882?src=c.1882.hamburger_menu_fly_out_banner, parent:None,
 ID:None, name:Nhà Cửa Đời Sống, url:https://tiki.vn/nha-cua-doi-song/c1883?src=c.1883.hamburger_menu_fly_out_banner, parent:None,
 ID:None, name:Hàng Tiêu Dùng - Thực Phẩm, url:

In [6]:
import re

# get_sub_categories() given a parent category
def get_sub_categories(parent_category, save_db=False):
    parent_url = parent_category.url
    result = []

    try:
        soup = get_url(parent_url)
        div_containers = soup.find_all('div', {'class':'list-group-item is-child'})
        for div in div_containers:
            name = div.a.text

            # replace more than 2 spaces with one space
            name = re.sub('\s{2,}', ' ', name)

            sub_url = TIKI_URL + div.a['href']
            cat = Category(name, sub_url, parent_category.cat_id)
            if save_db:
                cat.save_into_db()
            result.append(cat)
            conn.commit()
    except Exception as err:
        print('ERROR BY GET SUB CATEGORIES:', err)
    return result

In [7]:
# get_all_categories() given a list of main categories (This is a recusion function)
def get_all_categories(categories):
    if len(categories) == 0:
        return
    for cat in categories:
        sub_categories = get_sub_categories(cat, save_db=False)
        print(sub_categories)
        get_all_categories(sub_categories)

In [8]:
# get_all_categories(main_categories)

In [9]:
#Get the last layer in categories and put its products into a dataframe using pandas 

# This is the query that helps filter out the last layer of these categories 
SQL_Query = pd.read_sql_query( 
''' SELECT a.id, a.name, a.url, b.parent_id 
        FROM categories as a 
        LEFT JOIN categories as B 
        ON b.parent_id = a.id 
        WHERE b.parent_id is NULL''', conn)
# Create a dataframe based on the query and tiki-4.db
layer_last = pd.DataFrame(SQL_Query, columns=['id','name','url','parent_id'])
layer_last

,id,name,url,parent_id
0,17,Máy tính bảng (39)\n,https://tiki.vn/may-tinh-bang/c1794?src=c.1789...,None
1,18,Máy đọc sách (21)\n,https://tiki.vn/may-doc-sach/c28856?src=c.1789...,None
2,19,Điện thoại Smartphone (178)\n,https://tiki.vn/dien-thoai-smartphone/c1795?sr...,None
3,20,Điện thoại bàn (102)\n,https://tiki.vn/dien-thoai-ban/c8061?src=c.178...,None
4,21,Điện thoại phổ thông (100)\n,https://tiki.vn/dien-thoai-pho-thong/c1796?src...,None
...,...,...,...,...
2661,3227,GYM (28)\n,https://tiki.vn/gym/c13366?src=c.11312.hamburg...,None
2662,3228,Yoga (3)\n,https://tiki.vn/yoga/c13362?src=c.11312.hambur...,None
2663,3229,Ca nhạc - Phim - Kịch (2)\n,https://tiki.vn/ca-nhac-phim-kich/c11328?src=c...,None
2664,3230,Studio - Chụp ảnh (47)\n,https://tiki.vn/studio-chup-anh/c11330?src=c.1...,None


In [ ]:
# Create a table products in our db to insert products'info into.

def create_product_table():
    query = """
        CREATE TABLE IF NOT EXISTS 
        products(
            Id INTEGER PRIMARY KEY AUTOINCREMENT,
            Title VARCHAR(255),
            Tiki_Product_Id INTEGER,
            Price INTEGER,
            Link TEXT, 
            Image TEXT,
            Review VARCHAR(255),
            Cat_Id INTEGER,
            Discount TEXT,
            create_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
    """
    try:
        cur.execute(query)
        conn.commit()
    except Exception as err:
        print('ERROR BY CREAT TABLE',err)
create_product_table()

In [10]:
# Create a Product class to use later on for a loop
class Product:
    def __init__(self, title, tiki_product_id, price, url, img_url, review, cat_id, discount, id = None):
        self.id = id
        self.title = title
        self.tiki_product_id = tiki_product_id
        self.price = price
        self.url = url
        self.img_url =img_url
        self.review = review
        self.cat_id = cat_id
        self.discount = discount
        
    def __repr__(self):
        return f"ID:{self.id}, name: {self.title}, produc t_id: {self.tiki_product_id}, price: {self.price}, url: {self.url}, img_url: {self.img_url}, review: {self.review}, cat_id:{self.cat_id}"
    
    def save_into_db(self):
        query =   """
        INSERT INTO 
        products (Title, Tiki_Product_Id, Price, Link, Image, Review, Cat_Id,Discount)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?);
        """
        val = (self.title, self.tiki_product_id, self.price, self.url, self.img_url, self.review, self.cat_id, self.discount)
        try:
            cur.execute(query,val)
            self.id = cur.lastrowid
            conn.commit()
        except Exception as err:
            print ('ERROR BY INSERT:',err)

In [11]:

# Function to retrieve and parse the HTML code of a website 
def get_url(url):
    
    # Retrieve plain HTML code
    r = requests.get(url)  
    
    # Parse the plain content into structured one
    soup = BeautifulSoup(r.text, 'html.parser')
    
    return soup

# Function to actually get the products' info in one page.
def get_products(url, cat_id, save_db=False):
    
    # Get the parsed html code
    soup = get_url(url)
    # Sleep to avoid ip being blocked
    time.sleep(5)
    
    # Initialize empty 'results' list
    result = []
    
    # Find all products on this page and loop through each of them:
    for product in soup.find_all('div', {'class': 'product-item'}):
        
        # Extract the title of each product: 
        title = product.find('p', {'class': 'title'}).text.strip()
        
        # Extract product id  of each product: 
        tiki_product_id = product['data-seller-product-id']
        
        # Extract the final price of each product: 
        price = product.find('span',{'class':'final-price'}).text.split()[0][0:-1].replace('.','')
        
        # Extract the link that connect to each product: 
        url = 'https://tiki.vn' + product.a['href']
        
        # Extract the image of each product: 
        img_url = product.img['src']
        
        # Extract the review of each product: 
        review = product.find('p',{'class':'review'}).text.replace(')','').replace('(','')
        
        # Extract the discount value of each product(There are some products that dont have discount): 
        try:
              discount = product.find('p', {'class':'price-sale'}).text.split()[1]
        except:
              pass 
            
        # put these value into the Product class as an instance
        product_input = Product(title, tiki_product_id, price, url, img_url, review,cat_id,discount)
        
        #Saving and insert these products into databases using the product class method : save_into_db()
        if save_db:
            product_input.save_into_db()
        result.append(product_input)
    return result

In [ ]:
# For loop through the dataframe that now have turned into a numpy array : 
for row in layer_last.to_numpy():
    # Get these vales and turned them into string to help the database read them easier
    url = str(row[2])
    cat_id = str(row[0])
    name = str(row[1])
    # A nested for loop inside to help loop through all the pages that needs to be scraped:
    for i in range(23):
        url_cat = url+"&page="+str(i)
        # Using the function get_products defined above to get the data for one page
        result= get_products(url, cat_id ,save_db=False)
        # the loop will stop if there is no more result. 
        if len(result) == 0:
            break